## Deliverable 2. Create a Customer Travel Destinations Map.

In [19]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [20]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Florianopolis,BR,-27.6146,-48.5012,71.55,83,20,8.05,few clouds
1,1,Tera,NE,14.0078,0.7531,82.35,52,100,1.74,overcast clouds
2,2,Mubi,NG,10.2676,13.2644,72.52,88,100,2.80,light rain
3,3,Comonfort,MX,20.7167,-100.7667,79.29,47,100,7.78,overcast clouds
4,4,Thompson,CA,55.7435,-97.8558,90.88,23,100,2.89,overcast clouds


In [21]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [30]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Florianopolis,BR,-27.6146,-48.5012,71.55,83,20,8.05,few clouds
2,2,Mubi,NG,10.2676,13.2644,72.52,88,100,2.80,light rain
3,3,Comonfort,MX,20.7167,-100.7667,79.29,47,100,7.78,overcast clouds
7,7,Norman Wells,CA,65.2820,-126.8329,70.11,45,99,13.13,overcast clouds
9,9,Port-Gentil,GA,-0.7193,8.7815,73.94,81,100,11.56,overcast clouds
13,13,Avarua,CK,-21.2078,-159.7750,77.05,69,75,14.97,broken clouds
31,31,Enugu,NG,6.4402,7.4943,71.96,93,100,6.26,overcast clouds
33,33,Madang,PG,-5.2246,145.7966,75.67,92,99,3.09,overcast clouds
40,40,Puerto Ayora,EC,-0.7393,-90.3518,76.96,89,39,7.00,scattered clouds
42,42,Hambantota,LK,6.1241,81.1185,79.34,82,100,16.20,overcast clouds


In [31]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                200
City                   200
Country                199
Lat                    200
Lng                    200
Max Temp               200
Humidity               200
Cloudiness             200
Wind Speed             200
Current Description    200
dtype: int64

In [32]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                199
City                   199
Country                199
Lat                    199
Lng                    199
Max Temp               199
Humidity               199
Cloudiness             199
Wind Speed             199
Current Description    199
dtype: int64

In [33]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Florianopolis,BR,71.55,few clouds,-27.6146,-48.5012,
2,Mubi,NG,72.52,light rain,10.2676,13.2644,
3,Comonfort,MX,79.29,overcast clouds,20.7167,-100.7667,
7,Norman Wells,CA,70.11,overcast clouds,65.2820,-126.8329,
9,Port-Gentil,GA,73.94,overcast clouds,-0.7193,8.7815,
13,Avarua,CK,77.05,broken clouds,-21.2078,-159.7750,
31,Enugu,NG,71.96,overcast clouds,6.4402,7.4943,
33,Madang,PG,75.67,overcast clouds,-5.2246,145.7966,
40,Puerto Ayora,EC,76.96,scattered clouds,-0.7393,-90.3518,
42,Hambantota,LK,79.34,overcast clouds,6.1241,81.1185,


In [34]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [44]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Florianopolis,BR,71.55,few clouds,-27.6146,-48.5012,Hotel Boutique Quinta das Videiras
2,Mubi,NG,72.52,light rain,10.2676,13.2644,LASVEGAS BAR AND MOTEL
3,Comonfort,MX,79.29,overcast clouds,20.7167,-100.7667,El Kiosco Verde
7,Norman Wells,CA,70.11,overcast clouds,65.2820,-126.8329,Heritage Hotel
9,Port-Gentil,GA,73.94,overcast clouds,-0.7193,8.7815,Fengshui Residence
...,...,...,...,...,...,...,...
652,Oistins,BB,77.09,broken clouds,13.0667,-59.5333,Butterfly Beach Hotel
660,Moanda,GA,78.51,broken clouds,-1.5575,13.2178,Heliconia Stadium Moanda
664,Nabire,ID,75.18,overcast clouds,-3.3667,135.4833,Hotel Nusantara I
665,Nanchong,CN,77.27,overcast clouds,30.7951,106.0847,Marine Una Century Hotel


In [46]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"] != "")]
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Florianopolis,BR,71.55,few clouds,-27.6146,-48.5012,Hotel Boutique Quinta das Videiras
2,Mubi,NG,72.52,light rain,10.2676,13.2644,LASVEGAS BAR AND MOTEL
3,Comonfort,MX,79.29,overcast clouds,20.7167,-100.7667,El Kiosco Verde
7,Norman Wells,CA,70.11,overcast clouds,65.2820,-126.8329,Heritage Hotel
9,Port-Gentil,GA,73.94,overcast clouds,-0.7193,8.7815,Fengshui Residence
...,...,...,...,...,...,...,...
652,Oistins,BB,77.09,broken clouds,13.0667,-59.5333,Butterfly Beach Hotel
660,Moanda,GA,78.51,broken clouds,-1.5575,13.2178,Heliconia Stadium Moanda
664,Nabire,ID,75.18,overcast clouds,-3.3667,135.4833,Hotel Nusantara I
665,Nanchong,CN,77.27,overcast clouds,30.7951,106.0847,Marine Una Century Hotel


In [48]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [53]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [56]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))